<a href="https://www.kaggle.com/code/osmanacar/flo-rfm-analysis?scriptVersionId=187668054" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**FLO-RFM Analysis**
*  master_id: unique customer number
*  order_channel :the channel where orders are placed
*  last_order_channel : the shopping platform that utilized by the customer (Android, ios, Desktop, Mobile, Offline)
*  first_order_date : customer's first purchase date
*  last_order_date : customer's last purchase date
*  last_order_date_online : customer's last purchase date on online platform
*  last_order_date_offline : customer's last purchase date on offline platform
*  order_num_total_ever_online : total number of purchases by customers on the online platform
*  order_num_total_ever_offline : total number of purchases by customers on the offline platform
*  customer_value_total_ever_offline : total amount of purchases by customers on the offline platform
*  customer_value_total_ever_online : total amount of purchases by customers on the online platform
*  interested_in_categories_12 : Category list for the last 12 months

In [ ]:
import datetime as dt
import pandas as pd
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df = pd.read_csv("/kaggle/input/flo-dataset/flo_data_20k.csv")
df.head()

In [ ]:
df.info()
df.describe().T

In [ ]:
# We will create new variables for each customer's total amount and total number of purchases.
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]

In [ ]:
# If there is a variable that represents a date, we change the variable type**
all_of_date = df.columns[df.columns.str.contains("date")]
df[all_of_date] = df[all_of_date].apply(pd.to_datetime)

df.dtypes

In [ ]:
# Review of the number of the customers in order channels, the total number of products purchased and the total amount
df.groupby("order_channel").agg(
    {
        "master_id": "count",
        "order_num_total": "sum",
        "customer_value_total": "sum" 
    }
)

sns.countplot(x=df["order_channel"])

In [ ]:
# The top 10 customers who bring the highest revenue
df.sort_values(by="customer_value_total", ascending=False).head(10)

In [ ]:
# The top 10 customers who place the most orders
df.sort_values(by="order_num_total", ascending=False).head(10)

**Calculating RFM metrics**

In [ ]:
# We choose the analysis date 2 days after the last purchase date.
df["last_order_date"].max() # 2021-05-30
today_date = dt.datetime(2021, 6, 1)

In [ ]:
# We are creating new dataframe with customer_id, recency, frequency and monetary values.
rfm = pd.DataFrame()
rfm["customer_id"] = df["master_id"]
rfm["recency"] = df.apply(lambda x: (today_date - x["last_order_date"]).days, axis=1)
rfm["frequency"] = df["order_num_total"]
rfm["monetary"] = df["customer_value_total"]

rfm.head()

In [ ]:
# Calculating RFM scores**
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])

rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

rfm.head()

In [ ]:
# RFM scores define as segment
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map, regex=True)
rfm.head(20)

In [ ]:
# We are gonna calculate mean and count for recency, frequency, monetary
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

**Question 1**

FLO company wants to communicate with its customers about the new women's shoe brand it will add.

for example champions/loyal customers and KADIN category. 

after process we will add to new dataframe for customer id's

In [ ]:
# We add the categories column to the rfm dataframe.
rfm["interested_in_categories_12"] = df["interested_in_categories_12"].tolist()

rfm_for_female = rfm.loc[
    ((rfm["segment"] == "champions") | (rfm["segment"] == "loyal_customers")) &
    (rfm["interested_in_categories_12"].str.contains("KADIN") == True),
    ["segment", "interested_in_categories_12"]
]

rfm_for_female.head(15)


# We will add user ids to a new dataset.

# new_rfm_for_woman = pd.DataFrame()
# new_rfm_for_woman["new_customer_for_woman_category_id"] = rfm_for_woman.index
# new_rfm_for_woman.to_csv("new_customer_for_woman_category_id.csv")

In [ ]:
rfm_for_female.shape[0]

**Question 2**

FLO Company has planning %40 discount for ERKEK and COCUK category. 

They want to communicate past customers who should not be lost, at sleeping customers and new customers.

after process we will add to new dataframe for customer id's

In [ ]:
rfm_for_male_and_child = rfm.loc[(((rfm["segment"] == "cant_loose") |
                (rfm["segment"] == "hibernating") |
                (rfm["segment"] == "new_customers"))) &
              (((rfm["interested_in_categories_12"].str.contains("ERKEK") == True) |
                (rfm["interested_in_categories_12"].str.contains("COCUK") == True))),
                ["segment","interested_in_categories_12"]
            ]

rfm_for_male_and_child.head(20)

In [ ]:
rfm_for_male_and_child.shape[0]